<a href="https://colab.research.google.com/github/angelsoalmeida/imersao-dados-desafio-final/blob/main/Avaliando_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Preparando bases para avaliar os modelos**

In [ ]:
import pandas as pd
import numpy as np

#importando base experimento
dados = pd.read_csv('https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_experimentos.zip?raw=true', compression = 'zip')
#importando base resultado
dados_resultados = pd.read_csv('https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_resultados.csv?raw=true')

#efetuando alterações nas colunas
mapa = {'droga': 'composto'}
dados.rename(columns=mapa, inplace=True)

#Criação de coluna contendo a quantidade de mecanismos ativados por id
dados_resultados['n_moa'] = dados_resultados.drop('id', axis=1).sum(axis=1)

#Criação de coluna informando se para determinado id houve ou não ativação de algum mecanismo de ação
dados_resultados['ativo_moa'] = (dados_resultados['n_moa'] != 0)

#Efetuando merge dos dados, ou seja juntado as tabelas experimentos e resultados com base no id
dados_combinados = pd.merge(dados, dados_resultados[['id','n_moa', 'ativo_moa']], on='id')

#removendo colunas que não deverão ser analisadas pelos modelos
x = dados_combinados.drop(['id', 'n_moa', 'ativo_moa', 'composto'], axis=1)

#convertendo colunas de categoria em valores numerico, pois alguns modelos não conseguem trabalhar com strings
x = pd.get_dummies(x, columns=['tratamento', 'dose', 'tempo'])
y = dados_combinados['ativo_moa'] 

In [ ]:
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=376)

print("Quantidade de dados de treino: %d \nQuantidade de dados de teste: %d " % (len(x_treino), len(x_teste)))

Qunatidade de dados de treino: 19051 
Quantidade de dados de teste: 4763 


In [ ]:
print("Quantidade de dados de y_treino: %d \nQuantidade de dados de y_teste: %d " % (len(y_treino), len(y_teste)))

Quantidade de dados de y_treino: 19051 
Quantidade de dados de y_teste: 4763 


##**DummyClassifier**

Utilizando dummy somente para setar baseline de comparação, devido a sua natureza não é aconselhável utilizar em modelos reais.

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

#dummy sem parametros assume stratified como estratégia
modelo_dummy = DummyClassifier()
modelo_dummy.fit(x_treino, y_treino)
previsao_dummy = modelo_dummy.predict(x_teste)
print("Acuracia Dummy Stratified %.2f%%" %accuracy_score(y_teste, previsao_dummy))

modelo_dummy_mf = DummyClassifier('most_frequent')
modelo_dummy_mf.fit(x_treino, y_treino)
previsao_dummy_mf = modelo_dummy_mf.predict(x_teste)
print("Acuracia Dummy Most Frequent %.2f%%" %accuracy_score(y_teste, previsao_dummy_mf))

Acuracia Dummy Stratified 0.52%
Acuracia Dummy Most Frequent 0.61%


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


Conforme verificado o modelo DummyClassifier com a estrategia *most frequent *teve uma acurácia melhor que a do *stratified*, por isso usaremos o most frequent como baseline.




##**LogisticRegression**

In [ ]:
from sklearn.linear_model import LogisticRegression

modelo_rlogistica = LogisticRegression(max_iter=1000)
modelo_rlogistica.fit(x_treino, y_treino)

print("Acurácia do LogisticRegression %.2f%%" % modelo_rlogistica.score(x_teste, y_teste))

Acurácia do LogisticRegression 0.66%


##**LinearSVC**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

#setamos o parametro seed do random do numpy porque alguns modelos o utilizam
random = 5
np.random.seed(random)

modelo = LinearSVC()
modelo.fit(x_treino, y_treino)
previsoes = modelo.predict(x_teste)

acuracia = accuracy_score(y_teste, previsoes) * 100
print("A acurácia LinearSVC foi %.2f%%" % acuracia)

A acurácia foi 60.84%


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


##**DecisionTreeClassifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

modelo_arvore = DecisionTreeClassifier(max_depth=6)
modelo_arvore.fit(treino_x, y_treino)
previsoes = modelo_arvore.predict(treino_x)

acuracia = accuracy_score(y_treino, previsoes) * 100
print("A acuracia DecisionTreeClassifier do treino foi %.2f%%" % acuracia)


A acuracia DecisionTreeClassifier do treino foi 69.64%


###Verificando possibilidades de max depht para árvore de decisão

In [ ]:
#verificando possibilidades de max_depth para arvore de decisão
for random in range(5,10):
  np.random.seed(random)
  treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                          stratify = y)
  print("np.random.seed= %d" % random)

  for i in range(1,11):
    modelo = DecisionTreeClassifier(max_depth=i)
    modelo.fit(treino_x, treino_y)
    previsoes = modelo.predict(teste_x)

    acuracia = accuracy_score(teste_y, previsoes) * 100
    print("\nmax_depth %d" % i)
    print("\nA acurácia foi %.2f%%" % acuracia)

np.random.seed= 5

max_depth 1

A acurácia foi 68.39%

max_depth 2

A acurácia foi 68.39%

max_depth 3

A acurácia foi 68.53%

max_depth 4

A acurácia foi 68.56%

max_depth 5

A acurácia foi 68.66%

max_depth 6

A acurácia foi 68.51%

max_depth 7

A acurácia foi 68.54%

max_depth 8

A acurácia foi 68.59%

max_depth 9

A acurácia foi 68.12%

max_depth 10

A acurácia foi 68.00%
np.random.seed= 6

max_depth 1

A acurácia foi 68.39%

max_depth 2

A acurácia foi 68.39%

max_depth 3

A acurácia foi 68.83%

max_depth 4

A acurácia foi 68.64%

max_depth 5

A acurácia foi 68.89%

max_depth 6

A acurácia foi 68.74%

max_depth 7

A acurácia foi 68.68%

max_depth 8

A acurácia foi 68.42%

max_depth 9

A acurácia foi 67.53%

max_depth 10

A acurácia foi 67.35%
np.random.seed= 7

max_depth 1

A acurácia foi 68.95%

max_depth 2

A acurácia foi 68.95%

max_depth 3

A acurácia foi 69.05%

max_depth 4

A acurácia foi 68.98%

max_depth 5

A acurácia foi 69.21%

max_depth 6

A acurácia foi 69.21%

max_dep

Os testes não retornaram acuracia melhor do que os parametros originais portanto seguirei com os mesmos.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo_randomforest = RandomForestClassifier()
modelo_randomforest.fit(treino_x, treino_y)
modelo_randomforest.score(teste_x, teste_y)

0.6818945246892845

In [ ]:
print("A acuracia RandomForestClassifier foi %.2f%%" % modelo_randomforest.score(teste_x, teste_y))

A acuracia RandomForestClassifier foi 0.68%
